In [1]:
%pathsetup lib/python

"Updated sys path to include repository root and as well as subdirs ['lib/python']"

In [2]:
import numpy as np
import pandas as pd
#import rmm_util_accessor
import itertools
import time
import re
import grid

In [3]:
day = 11
#data = "125 17"
data = "20 82084 1650 3 346355 363 7975858 0"
data = data.split(" ")

In [4]:
data

['20', '82084', '1650', '3', '346355', '363', '7975858', '0']

# Rules
If the stone is engraved with the number 0, it is replaced by a stone engraved with the number 1.

If the stone is engraved with a number that has an even number of digits, it is replaced by two stones. 
The left half of the digits are engraved on the new left stone, and the right half of the digits are engraved on the new right stone. 
(The new numbers don't keep extra leading zeroes: 1000 would become stones 10 and 0.)

If none of the other rules apply, the stone is replaced by a new stone; the old stone's number multiplied by 2024 is engraved on the new stone.

In [5]:
def blink(x):
    if x=='0':
        new_x = ['1']
    elif len(x)%2==0:
        new_x = [ str(int(x[:len(x)//2])), str(int(x[len(x)//2:])) ]
    else:
        new_x = [ str(int(x) * 2024) ]
    return new_x

In [ ]:
%%time
supermap = {}
origdata = "20 82084 1650 3 346355 363 7975858 0"
origdata = origdata.split(" ")
data = origdata.copy()
new = set(data.copy())
N = 75
for i in range(N):   
    print(f"Working blink #{i}")
    working = new.copy()
    new = set()
    for x in working:
        if x not in supermap:
            result = blink(x)
            supermap[x] = result
            new.update(result)
    #print(data)

In [7]:
len(supermap)

3972

In [ ]:
%%time
# The keys in the supermap are the only numbers I'll ever need to know for this
# So do a 1 blink step for each first
oneblink = {}
for val in sorted(supermap.keys()):
    data = blink(val)
    print(f"{val} --> {data}")
    oneblink[val] = data.copy()

In [9]:
oneblink['20']

['2', '0']

In [10]:
%%time
num = {}  # dictionary of counters for each element
for d in origdata:
    num[d] = 1
working = num.copy()

rounds = 75
for i in range(rounds):
    #print("--------")
    for stone, n in num.items():
        newstones = oneblink[stone]
        #print(f"Loop {i}, Stone: {stone} x {n} -->", newstones)
        working[stone] -= n
        if working[stone] == 0:
            del working[stone]
        for s in newstones:
            if s in working:
                working[s] += n  
            else:
                working[s] = n   
    num = working.copy()   
    

CPU times: user 31.2 ms, sys: 0 ns, total: 31.2 ms
Wall time: 74.7 ms


In [11]:
sum(num.values())

205913561055242

In [119]:
working

{'8096': 2, '1': 1, '16192': 1, '2': 2, '0': 1, '4': 1}

# All junk below here

In [67]:
%%time
#for x in data:
if True:
    x = '20'
    nextlist = [x]
    for n in range(2):
        tmp = []
        for val in nextlist:
            tmp.extend(oneblink[val])
        nextlist = tmp.copy()   

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 29.1 µs


In [68]:
len(nextlist), len(set(nextlist))

(2, 2)

In [69]:
ser = pd.Series(nextlist)

In [70]:
ser.value_counts().sort_values(ascending=False)

4048    1
1       1
Name: count, dtype: int64

In [11]:
thangs = []
for x in data:
    thangs.append([x])

In [12]:
thangs

[['20'], ['82084'], ['1650'], ['3'], ['346355'], ['363'], ['7975858'], ['0']]

In [ ]:
%%time
thangs = {}
for i,x in enumerate(data[:1]):
    thangs[i] = [x]
newthangs = {}
for n in range(40):
    print(f"***** n={n}")
    for i,thing in enumerate(thangs.values()):
        #print(f"### i={i}, {thing}")
        newthangs[i] = []
        for x in thing:
            #print(f"x={x}")
            newthangs[i].extend(supermap[x])
            #print(newthangs[i])
    thangs = newthangs.copy()

In [ ]:
thangs

In [24]:
list(thangs.keys())

[0]

In [13]:
def fastblink(this_data):
    ans = []
    for x in this_data:
        if x in oneblink:
            ans.extend(oneblink[x])
        else:
            ans.extend(blink(x))
    return ans

In [ ]:
%%time
data = "125 17"
data = data.split(" ")
for i in range(35):
    #print(f"Working blink #{i}")
    data = fastblink(data)
    #print(len(data))

In [ ]:
%%time
data = "1 2 3"
data = data.split(" ")
for i in range(25):
    print(f"Working blink #{i}")
    data = blink(data)
    print(data)